In [11]:
####################
## load libraries ##
####################
import numpy as np
from sklearn.metrics import roc_auc_score
np.random.seed(1234567)

In [12]:
############################
## Toy Dataset Simulation ##
############################
def simulate_dataset(n=10000, features=10):
    
    np.random.seed(1081565)
    ## divide number of features by 2
    p = int(features/2)
    ## specify intercept column
    X_intercept = np.ones((n,1))
    ## create p features over the reals
    X_1 = np.random.uniform(-2, 2, size=(n, p))
    ## create p features with support {0,1}
    X_2 = np.random.randint(low=0, high=2, size=(n, p))
    ## concate intercept, X1, and X2
    X = np.concatenate((X_intercept, X_1, X_2), axis=1)
    ## save the original "Design Matrix" of features to return later
    D = np.concatenate((X_1, X_2), axis=1)
    
    ## create 10 additional features
    X_11 = (X[:, 1] * X[:, 1]).reshape((n, 1))
    X_12 = (X[:, 3] * X[:, 3]).reshape((n, 1))
    X_13 = (X[:, 2] * X[:, 4]).reshape((n, 1))
    X_14 = (X[:, 2] * X[:, 8]).reshape((n, 1))
    X_15 = (X[:, 7] * X[:, 9]).reshape((n, 1))
    X_16 = np.cos(X[:, 4]).reshape((n, 1))
    X_17 = np.sin(X[:, 2]).reshape((n, 1))
    X_18 = (X_16 * X_11).reshape((n, 1))
    X_19 = (X_17 * X_12).reshape((n, 1))
    X_20 = (X_16 * X_16).reshape((n, 1))
    X = np.concatenate((X, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19, X_20), axis=1)
    del X_intercept, X_1, X_2, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19, X_20
    
    ## specify Betas
    Beta = np.random.uniform(-1, 1, size=((2*features)+1, 1))

    ## Fit the outcome Y
    theta = np.dot(X, Beta)
    prob = (1 / (1 + np.exp(-theta)))
    
#    ## specify categorical {0,1} outcome
#    Y = np.zeros((n, 1))
#    for i in range(0, n):
#        Y[i, 0] = np.random.choice([0, 1], size=1, replace=True, p=[1-prob[i, 0], prob[i, 0]])    
    ## specify categorical {0,1} outcome
    Y = np.zeros((n, 1))
    for i in range(0, n):
        Y[i, 0] = np.random.choice([0, 1], size=1, replace=True, p=[1-prob[i, 0], prob[i, 0]])  
 
    ## return outcome Y and original Design Matrix D
    return(Y, D)

In [13]:
##############################
## Neural Network Functions ##
##############################
def Normalize_Design_Matrix(D):
    D_means = D.mean(axis=0, keepdims=True)
    D_std = D.std(axis=0, keepdims=True)
    D_normalized = (D-D_means) / D_std
    return(D_normalized, D_means, D_std)

def initialize_Beta(p1, p2=1):
    Beta = np.random.uniform(-1, 1, size=(p1, p2))
    return(Beta)

def initialize_b(p2=1):
    b = np.random.uniform(-1, 1, size=(1, p2))
    return(b)

def sigmoid(theta):
    Y = 1 / (1 + np.exp(-theta))
    return(Y)
    
def sigmoid_derivative(Z):
    Y = sigmoid(Z) * (1-sigmoid(Z))
    return(Y)
    
def softmax(theta):
    Y = np.exp(theta)
    Y = Y / Y.sum(axis=1).reshape((Y.shape[0],1))   
    return(Y)
    
def softmax_derivative(Z):
    Y = softmax(Z) * (1-softmax(Z))
    return(Y)
    
def tanh(theta):
    Y = (np.exp(theta) - np.exp(-theta)) / (np.exp(theta) + np.exp(-theta))
    return(Y)

def tanh_derivative(Z):
    Y = 1 - (tanh(Z)*tanh(Z))
    return(Y)

def reLu(theta):
    Y = theta.copy()
    Y[Y<0]=0
    return(Y)
    
def reLu_derivative(Z):
    return(1)
    
def identity(theta):
    Y = theta.copy()
    return(Y)
    
def identity_derivative(Z):
    return(1)
   

def specify_mini_batch_size(n, mini_batch_size):
    if(n % mini_batch_size == 0):
        start = (n//mini_batch_size)*[None]
        end = (n//mini_batch_size)*[None]
        m = (n//mini_batch_size)*[None]
        for i in range(0, (n//mini_batch_size)):
            start[i] = i*mini_batch_size
            end[i] = (i+1)*mini_batch_size
            m[i] = mini_batch_size
        del i
    else:
        start = int(np.ceil(n/mini_batch_size))*[None]
        end = int(np.ceil(n/mini_batch_size))*[None]
        m = int(np.ceil(n/mini_batch_size))*[None]
        for i in range(0, (n//mini_batch_size)):
            start[i] = i*mini_batch_size
            end[i] = (i+1)*mini_batch_size
            m[i] = mini_batch_size
        start[i+1] = end[i]
        end[i+1] = n
        m[i+1] = end[i+1] - start[i+1]
        del i
    return(start, end, m)



    

class NeuralNetwork:
    def __init__(self, Y, D):
        self.n = D.shape[0]
        self.Y = Y
        self.D = D
        self.D_normalized, self.D_means, self.D_std = Normalize_Design_Matrix(self.D)
    
    def specify_layers(self, p, activations):
        self.p = p
        self.activations = activations
        self.Betas = {}
        self.bs = {}
        self.X = {}
        self.Z = {}
        self.Y_hat = None
        self.dJ_dZ = {}
        self.dJ_dB = {}
        self.dJ_db = {}
        for i in range(0, len(self.p)):
            
            if(i<len(self.p)-1):
                self.Betas['B'+str(i+1)] = initialize_Beta(p1=self.p[i], p2=self.p[i+1])
                self.bs['b'+str(i+1)] = initialize_b(p2=self.p[i+1])
            else:
                self.Betas['B'+str(i+1)] = initialize_Beta(p1=self.p[i], p2=self.Y.shape[1])
                self.bs['b'+str(i+1)] = initialize_b(p2=self.Y.shape[1])
                
            self.X['X'+str(i+1)] = None
            self.dJ_dZ['dJ_dZ'+str(i+1)] = None
            self.dJ_dB['dJ_dB'+str(i+1)] = None
            self.dJ_db['dJ_db'+str(i+1)] = None
                
            
    def forward_dropout(self, j, dropout_probability):
        self.X['X'+str(j)] = (1/(1-dropout_probability)) * self.X['X'+str(j)] * np.random.binomial(1, (1-dropout_probability), size=(self.X['X'+str(j)].shape[0], self.X['X'+str(j)].shape[1]))
        
        
    def forward_Z(self, j):
        self.Z['Z'+str(j)] = np.dot(self.X['X'+str(j)], self.Betas['B'+str(j)]) + self.bs['b'+str(j)]
    
    
    def forward_X(self, j):
        activation = self.activations[j-1]
        if(activation=='reLu'):
            self.X['X'+str(j+1)] = reLu(self.Z['Z'+str(j)])
        if(activation=='sigmoid'):
            self.X['X'+str(j+1)] = sigmoid(self.Z['Z'+str(j)])
        if(activation=='tanh'):
            self.X['X'+str(j+1)] = tanh(self.Z['Z'+str(j)])
        if(activation=='softmax'):
            self.X['X'+str(j+1)] = softmax(self.Z['Z'+str(j)])
        if(activation=='identity'):
            self.X['X'+str(j+1)] = identity(self.Z['Z'+str(j)])
    
    
    def forward_Y(self):
        activation = self.activations[-1]
        if(activation=='reLu'):
            self.Y_hat = reLu(self.Z['Z'+str(len(self.Z))])
        if(activation=='sigmoid'):
            self.Y_hat = sigmoid(self.Z['Z'+str(len(self.Z))])
        if(activation=='tanh'):
            self.Y_hat = tanh(self.Z['Z'+str(len(self.Z))])
        if(activation=='softmax'):
            self.Y_hat = softmax(self.Z['Z'+str(len(self.Z))])
        if(activation=='identity'):
            self.Y_hat = identity(self.Z['Z'+str(len(self.Z))])

    
    def calculate_loss_function(self, Y_mini, i, m, k):
        activation = self.activations[-1]
        if(activation=='sigmoid' or activation=='softmax'):
            self.J = (1/m[i])*(-np.dot(Y_mini.T, np.log(self.Y_hat)) - np.dot((1-Y_mini).T, np.log(1-self.Y_hat))) 
        if(activation=='identity' or activation=='softmax'):
            self.J = (1/m[i])*np.dot((Y_mini-self.Y_hat).T, (Y_mini-self.Y_hat))
        if(i==(len(m)-1)):
            print('current loss on epoch ' + str(k+1) + ' is: ' + str(self.J.sum()))
            
    
    def backward_dJ_dZ(self, Y_mini, j):
        activation = self.activations[j-1]
        if(j==len(self.Z)):
            if(activation=='sigmoid'):
                self.dJ_dZ['dJ_dZ'+str(len(self.dJ_dZ))] = sigmoid(self.Z['Z'+str(len(self.Z))]) - Y_mini
            if(activation=='reLu'):
                self.dJ_dZ['dJ_dZ'+str(len(self.dJ_dZ))] = reLu(self.Z['Z'+str(len(self.Z))]) - Y_mini
            if(activation=='tanh'):
                self.dJ_dZ['dJ_dZ'+str(len(self.dJ_dZ))] = tanh(self.Z['Z'+str(len(self.Z))]) - Y_mini
            if(activation=='softmax'):
                self.dJ_dZ['dJ_dZ'+str(len(self.dJ_dZ))] = softmax(self.Z['Z'+str(len(self.Z))]) - Y_mini
            if(activation=='identity'):
                self.dJ_dZ['dJ_dZ'+str(len(self.dJ_dZ))] = identity(self.Z['Z'+str(len(self.Z))]) - Y_mini
        if(j<len(self.Z)):
            if(activation=='sigmoid'):
                self.dJ_dZ['dJ_dZ'+str(j)] = np.dot(self.dJ_dZ['dJ_dZ'+str(j+1)], self.Betas['B'+str(j+1)].T) * sigmoid_derivative(self.Z['Z'+str(j)])
            if(activation=='reLu'):
                self.dJ_dZ['dJ_dZ'+str(j)] = np.dot(self.dJ_dZ['dJ_dZ'+str(j+1)], self.Betas['B'+str(j+1)].T) * reLu_derivative(self.Z['Z'+str(j)])
            if(activation=='tanh'):
                self.dJ_dZ['dJ_dZ'+str(j)] = np.dot(self.dJ_dZ['dJ_dZ'+str(j+1)], self.Betas['B'+str(j+1)].T) * tanh_derivative(self.Z['Z'+str(j)])
            if(activation=='softmax'):
                self.dJ_dZ['dJ_dZ'+str(j)] = np.dot(self.dJ_dZ['dJ_dZ'+str(j+1)], self.Betas['B'+str(j+1)].T) * softmax_derivative(self.Z['Z'+str(j)])
            if(activation=='identity'):
                self.dJ_dZ['dJ_dZ'+str(j)] = np.dot(self.dJ_dZ['dJ_dZ'+str(j+1)], self.Betas['B'+str(j+1)].T) * identity_derivative(self.Z['Z'+str(j)])


    def backward_dJ_dB(self, j):
        self.dJ_dB['dJ_dB'+str(j)] = np.dot(self.X['X'+str(j)].T, self.dJ_dZ['dJ_dZ'+str(j)])
        
    
    def backward_dJ_db(self, j, i, m):
        self.dJ_db['dJ_db'+str(j)] = np.dot(np.ones((1, m[i])), self.dJ_dZ['dJ_dZ'+str(j)])
    
    
    def update_Betas(self, j, alpha):
        self.Betas['B'+str(j)] = self.Betas['B'+str(j)] - (alpha*self.dJ_dB['dJ_dB'+str(j)])
        
        
    def update_bs(self, j, alpha):
        self.bs['b'+str(j)] = self.bs['b'+str(j)] - (alpha*self.dJ_db['dJ_db'+str(j)])
    
    
    
    def fit(self, mini_batch_size, epochs=1, alpha=0.001, drop_out=False, dropout_probability=0):
        ###########################################################
        ## specify coordinates of minibatch sizes for each epoch ##
        ###########################################################
        start, end, m = specify_mini_batch_size(self.n, mini_batch_size)
        
        for k in range(0, epochs):     ## loop over the epochs
            for i in range(0,len(m)):           ## loop over the minibatches    
                Y_mini = self.Y[start[i]:end[i],:]
                
                ###################
                ## forward pass: ##
                ###################
                for j in range(1, len(self.X)):                    
                    ## normalize the input layer
                    if(j==1):
                        self.X['X'+str(j)] = self.D_normalized[start[i]:end[i],:]                    
                    ## initiate dropout regularization (if needed)
                    if(drop_out):
                        self.forward_dropout(j, dropout_probability)                    
                    ## calculate Z of current layer
                    self.forward_Z(j)                    
                    ## calculate X of next layer
                    self.forward_X(j)
                ## calculate Z of final layer
                self.forward_Z(j+1)
                ## calculate output layer Y
                self.forward_Y()

            
                ###########################################
                ## print current value of loss function: ##
                ###########################################
                self.calculate_loss_function(Y_mini, i, m, k)
                
            
                #####################
                ## backwards pass: ##
                #####################
                for j in range(len(self.Z), 0, -1):
                    ## calculate the gradient dJ_dTheta
                    self.backward_dJ_dZ(Y_mini, j)
                
                for j in range(len(self.X), 0, -1):
                    ## calculate the gradient dJ_dB
                    self.backward_dJ_dB(j)
                    ## calculate the gradient dJ_db
                    self.backward_dJ_db(j, i, m)

            
                ##################################
                ## update coefficient estimates: ##
                ##################################
                for j in range(len(self.Betas), 0, -1):
                    ## update Betas
                    self.update_Betas(j, alpha)
                    ## update intercets b
                    self.update_bs(j, alpha)
                                    
                del Y_mini

In [14]:
## simulate dataset
Y, D = simulate_dataset()

## initialize Neural Network class object
NN = NeuralNetwork(Y, D)

## fit the Neural Network
NN.specify_layers(p=[NN.D.shape[1], 100, 85, 80, 70, 60], activations=['reLu', 'sigmoid', 'reLu', 'sigmoid', 'reLu', 'sigmoid'])
NN.fit(mini_batch_size=10000, epochs=1000, alpha=0.000001, drop_out=False, dropout_probability=0.02)

current loss on epoch 1 is: 4.626386947606572
current loss on epoch 2 is: 2.643770386910068
current loss on epoch 3 is: 1.7559048935000692
current loss on epoch 4 is: 1.5914338785783473
current loss on epoch 5 is: 1.548000215292208
current loss on epoch 6 is: 1.5153298546522171
current loss on epoch 7 is: 1.48561193852702
current loss on epoch 8 is: 1.4580856182311097
current loss on epoch 9 is: 1.4324135211750917
current loss on epoch 10 is: 1.4083734564134422
current loss on epoch 11 is: 1.3857579065175314
current loss on epoch 12 is: 1.36429459841152
current loss on epoch 13 is: 1.3439364704232784
current loss on epoch 14 is: 1.324690879342278
current loss on epoch 15 is: 1.306507920300551
current loss on epoch 16 is: 1.2893254785222155
current loss on epoch 17 is: 1.273058905650284
current loss on epoch 18 is: 1.257677193746075
current loss on epoch 19 is: 1.2431193999854653
current loss on epoch 20 is: 1.2293265166495182
current loss on epoch 21 is: 1.2162722489950337
current loss

current loss on epoch 172 is: 0.7309353474806641
current loss on epoch 173 is: 0.7299777771868312
current loss on epoch 174 is: 0.7290285775604898
current loss on epoch 175 is: 0.7280863649699785
current loss on epoch 176 is: 0.7271510144412136
current loss on epoch 177 is: 0.7262245964367203
current loss on epoch 178 is: 0.725305668919695
current loss on epoch 179 is: 0.7243959835707796
current loss on epoch 180 is: 0.7234956963743926
current loss on epoch 181 is: 0.7226038863996064
current loss on epoch 182 is: 0.7217185617112447
current loss on epoch 183 is: 0.7208408473289543
current loss on epoch 184 is: 0.7199674181945331
current loss on epoch 185 is: 0.719102544412258
current loss on epoch 186 is: 0.7182463842286041
current loss on epoch 187 is: 0.717396001219208
current loss on epoch 188 is: 0.7165516383139562
current loss on epoch 189 is: 0.71571148758733
current loss on epoch 190 is: 0.7148780687196837
current loss on epoch 191 is: 0.7140543375034272
current loss on epoch 192

current loss on epoch 340 is: 0.6379490337951794
current loss on epoch 341 is: 0.6376236737394302
current loss on epoch 342 is: 0.6372985377210529
current loss on epoch 343 is: 0.6369750302630067
current loss on epoch 344 is: 0.6366522838675115
current loss on epoch 345 is: 0.6363302795656557
current loss on epoch 346 is: 0.6360103545598049
current loss on epoch 347 is: 0.6356928339745902
current loss on epoch 348 is: 0.6353758818503993
current loss on epoch 349 is: 0.6350608256196655
current loss on epoch 350 is: 0.6347484526921029
current loss on epoch 351 is: 0.6344379203148586
current loss on epoch 352 is: 0.634129475364104
current loss on epoch 353 is: 0.6338223624342422
current loss on epoch 354 is: 0.6335165242568773
current loss on epoch 355 is: 0.6332117819606065
current loss on epoch 356 is: 0.6329084215261732
current loss on epoch 357 is: 0.632605875295375
current loss on epoch 358 is: 0.6323045462375314
current loss on epoch 359 is: 0.6320036485831687
current loss on epoch 

current loss on epoch 508 is: 0.5981008328122119
current loss on epoch 509 is: 0.5979352903860717
current loss on epoch 510 is: 0.5977702488288232
current loss on epoch 511 is: 0.5976058240177239
current loss on epoch 512 is: 0.5974422955900252
current loss on epoch 513 is: 0.5972786161013853
current loss on epoch 514 is: 0.5971155506768506
current loss on epoch 515 is: 0.5969534346129752
current loss on epoch 516 is: 0.5967923476250533
current loss on epoch 517 is: 0.5966326373462701
current loss on epoch 518 is: 0.5964736148892674
current loss on epoch 519 is: 0.5963156115227479
current loss on epoch 520 is: 0.5961579626811498
current loss on epoch 521 is: 0.5959999364244719
current loss on epoch 522 is: 0.5958420131085002
current loss on epoch 523 is: 0.5956844277550355
current loss on epoch 524 is: 0.5955267954357308
current loss on epoch 525 is: 0.5953700526862972
current loss on epoch 526 is: 0.595214156485036
current loss on epoch 527 is: 0.5950579596311291
current loss on epoch

current loss on epoch 676 is: 0.5771878732586992
current loss on epoch 677 is: 0.577094158155298
current loss on epoch 678 is: 0.5770005814668162
current loss on epoch 679 is: 0.5769074142782499
current loss on epoch 680 is: 0.5768140728616684
current loss on epoch 681 is: 0.5767203882913581
current loss on epoch 682 is: 0.5766270559583129
current loss on epoch 683 is: 0.576533930977966
current loss on epoch 684 is: 0.5764412139451051
current loss on epoch 685 is: 0.5763483311618735
current loss on epoch 686 is: 0.5762553213446219
current loss on epoch 687 is: 0.5761625479231589
current loss on epoch 688 is: 0.5760695197176791
current loss on epoch 689 is: 0.5759758771569016
current loss on epoch 690 is: 0.5758819329811077
current loss on epoch 691 is: 0.5757874965144282
current loss on epoch 692 is: 0.5756938390747952
current loss on epoch 693 is: 0.5755998656824054
current loss on epoch 694 is: 0.575505587495649
current loss on epoch 695 is: 0.5754110093015051
current loss on epoch 6

current loss on epoch 844 is: 0.5641084361347496
current loss on epoch 845 is: 0.5640448096153974
current loss on epoch 846 is: 0.5639811875638381
current loss on epoch 847 is: 0.5639170718078856
current loss on epoch 848 is: 0.5638525580365975
current loss on epoch 849 is: 0.5637881073910911
current loss on epoch 850 is: 0.5637239925492235
current loss on epoch 851 is: 0.5636598852332934
current loss on epoch 852 is: 0.5635956205175897
current loss on epoch 853 is: 0.5635306478641546
current loss on epoch 854 is: 0.5634651808805033
current loss on epoch 855 is: 0.5633999442964416
current loss on epoch 856 is: 0.5633349970499145
current loss on epoch 857 is: 0.5632703203921767
current loss on epoch 858 is: 0.5632059103483844
current loss on epoch 859 is: 0.563140853003523
current loss on epoch 860 is: 0.5630763556347281
current loss on epoch 861 is: 0.5630125354084595
current loss on epoch 862 is: 0.5629491200869415
current loss on epoch 863 is: 0.5628856954199947
current loss on epoch

In [15]:
######################################
## Area Under the Curve calculation ##
######################################
Y_hat = NN.Y_hat
print(roc_auc_score(Y, Y_hat))

0.7937240335755801
